## Daily emailed weather forecasts

Like many others, my brain does not function well in the morning pre-coffee and so living in NYC with its tempermental weather I would inevitably get caught out either by rain or a sharp temperature drop. I could check the weather every morning but who remembers to do that? Instead I tried to find a way to insert a weather forecast into my normal routine in a way I couldn't miss. I thought about my morning routine and decided on two ways that I could inject a forecast without having to change my behaviour.  

1. Set up a daily email that includes both a easy to understand subject, and a quick reference chart. If you set your own email as a VIP email address in your iPhone mail app, this will mean that the subject will show as a lock screen notification, giving you a weather forecast if you even glance at the front of your phone. I often check the time on my phone in the morning, so I'll see the forecast immediately. 

2. In the time window during which I'm preparing to leave the house (7:45-8:15am), use color changing light bulbs to change the color of my living room lights to blue if it will rain, meaning I can't miss a warning to pick up my umbrella. 



In [8]:
import requests
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import smtplib

from os.path import basename
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.utils import formatdate
from creds import cdict

## Note I have a creds module in my pythonpath where I store any 
## sensitive information that can't be added to this repository, 
## it just takes the form of a single dictionary cdict

#### Exploring the forecast.io API and using it to generate a daily personal email

For most any api call, I often write a wrapper to handle authentication and remembering endpoints and the query structure, so I can interact with it in a simpler way. 

In [6]:
def get_raw_weather(lat='40.674766',lon='-73.978959'):
    '''Simple requests wrapper to allow retrival of the latest weather forecast from the forecast.io API'''
    data = requests.get(f"https://api.forecast.io/forecast/{cdict['forecastio_api_key']}/{lat},{lon}").json()
    return data

In [9]:
raw_data = get_raw_weather()

Let's have a quick look at the data we get back from this API call, for more details see https://darksky.net/dev/docs

In [20]:
for subkey in raw_data.keys():
    print(f'{subkey}: {type(raw_data[subkey])}') 

latitude: <class 'float'>
longitude: <class 'float'>
timezone: <class 'str'>
currently: <class 'dict'>
minutely: <class 'dict'>
hourly: <class 'dict'>
daily: <class 'dict'>
alerts: <class 'list'>
flags: <class 'dict'>
offset: <class 'int'>


This basically consists of some metadata about the call we made (lat, lon, timezone), the forecasts at various time resolutions (currently, minutely, hourly and daily), some alerts and flags that apply to the data and a deprectated offset, that should be ignored and you should use timezone instead. 

For our purposes we will be focusing on the hourly forecast, so let's extract that and clean it up a bit

In [32]:
hourly_df = pd.DataFrame.from_dict(raw_data['hourly']['data'])
hourly_df.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,28.09,0.89,29.11,0.91,snow,259.39,0.186,0.0231,0.30,snow,1012.25,Snow,31.35,1512838800,1,0.00,347,6.18,3.36
1,27.46,0.93,30.73,0.93,snow,258.95,0.215,0.0282,0.50,snow,1011.08,Snow,32.54,1512842400,1,0.00,351,8.94,5.29
2,28.02,0.98,31.08,0.88,snow,258.81,0.347,0.0456,0.62,snow,1010.01,Snow,34.26,1512846000,1,1.09,359,11.88,7.31
3,28.39,1.00,30.94,0.84,snow,258.92,0.316,0.0416,0.66,snow,1009.20,Snow,35.24,1512849600,0,3.79,4,13.97,8.72
4,28.09,1.00,30.94,0.84,snow,259.97,0.281,0.0370,0.63,snow,1008.57,Snow,35.23,1512853200,0,4.41,5,15.12,9.28


Wonderful we now have a pandas dataframe containing a row of weather forecast per hour. Our only problem is the time column is stored in UNIX/POSIX time, but thankfully the built in datetime library can fix that pretty quick for us:


In [33]:
hourly_df.time = hourly_df.time.apply(datetime.datetime.fromtimestamp)
hourly_df.set_index('time',inplace=True)
print(f'We have {len(hourly_df)} rows of weather data')
hourly_df.head()

We have 49 rows of weather data


,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
time,,,,,,,,,,,,,,,,,,
2017-12-09 12:00:00,28.09,0.89,29.11,0.91,snow,259.39,0.186,0.0231,0.30,snow,1012.25,Snow,31.35,1,0.00,347,6.18,3.36
2017-12-09 13:00:00,27.46,0.93,30.73,0.93,snow,258.95,0.215,0.0282,0.50,snow,1011.08,Snow,32.54,1,0.00,351,8.94,5.29
2017-12-09 14:00:00,28.02,0.98,31.08,0.88,snow,258.81,0.347,0.0456,0.62,snow,1010.01,Snow,34.26,1,1.09,359,11.88,7.31
2017-12-09 15:00:00,28.39,1.00,30.94,0.84,snow,258.92,0.316,0.0416,0.66,snow,1009.20,Snow,35.24,0,3.79,4,13.97,8.72
2017-12-09 16:00:00,28.09,1.00,30.94,0.84,snow,259.97,0.281,0.0370,0.63,snow,1008.57,Snow,35.23,0,4.41,5,15.12,9.28


Wonderful now we can see that we have hourly forecasts for the next two days starting with the current hour. 

In [ ]:
def doi_func():
    ''' Day of Interest '''
    now = datetime.datetime.now()
    if now.hour < 20:
        return now.date().strftime('%Y-%m-%d')
    else:
        return (now.date() + datetime.timedelta(1)).strftime('%Y-%m-%d')